 ## Loading Libraries 

In [1]:
import numpy as np
import math
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from scipy.sparse import csc_matrix,lil_matrix,csr_matrix,bsr_matrix,diags,coo_matrix,dok_matrix
from scipy.sparse.linalg import spsolve_triangular,spsolve
from sksparse.cholmod import cholesky
from sklearn.datasets import load_svmlight_file
from sklearn.datasets.samples_generator import make_swiss_roll
%load_ext Cython

## Reading data

### 1- Reading datasets in txt format 

In [13]:
X = np.loadtxt('skin.txt', skiprows=1)

#splitting
n_train = 108783
x_tr = X[:n_train,:-1]
y_tr = X[:n_train,-1]
x_tes = X[n_train:,:-1]
y_tes = X[n_train:,-1]


### 2- Reading datasets in LibSVM format

In [6]:
#x_tr,y_tr = load_svmlight_file("covtype.libsvm.binary.scale (1)")
#x_tr, x_tes, y_tr, y_tes = train_test_split(
#    x_tr, y_tr, test_size=0.33, random_state=88)

x_tr,y_tr = load_svmlight_file("shuttle.scale.txt")
x_tes,y_tes = load_svmlight_file("shuttle.scale.t")
x_tes = x_tes.toarray()
x_tr = x_tr.toarray()



### 3- Reading artifical datasets

In [2]:
# Generate data (swiss roll dataset)
n_samples = 100000
noise = 0.05
X1, _ = make_swiss_roll(n_samples, noise)
X2, _ = make_swiss_roll(n_samples, noise)
X3, _ = make_swiss_roll(n_samples, noise)

# Shrink second dimension
X1[:, 1] *= .1
X2[:, 1] *= .1
X3[:, 1] *= .1

def rotation(theta, R = np.zeros((3,3))):
    cx,cy,cz = np.cos(theta)
    sx,sy,sz = np.sin(theta)
    R.flat = (cx*cz - sx*cy*sz, cx*sz + sx*cy*cz, sx*sy,
        -sx*cz - cx*cy*sz, -sx*sz + cx*cy*cz,
        cx*sy, sy*sz, -sy*cz, cy)
    return R

R2 = rotation((0,np.pi/4,np.pi/4))
R3 = rotation((np.pi/3,np.pi/3,0))

X= np.concatenate((X1,2+X2.dot(R2),4+X3.dot(R3)))
#X= np.concatenate((X1,2+X2.dot(R2))

y1 = np.ones((n_samples))
y2 = 2*np.ones((n_samples))
y3 = 3*np.ones((n_samples))
Y = np.concatenate((y1,y2,y3))
#Y = np.concatenate((y1,y2))

x_tr, x_tes, y_tr, y_tes = train_test_split(
    X, Y, test_size=0.33, random_state=88)

scaler = StandardScaler()
scaler.fit(x_tr)
x_tr = scaler.transform(x_tr)
x_tes = scaler.transform(x_tes)


## Specifying Parameters

In [19]:
## # .... parameters
sigma = 0.002
gamma = 1/(2*(sigma**2))
mu= 1/(3*sigma);
m = x_tr.shape[0]
n = x_tr.shape[1]

# classes
classes = np.unique(y_tr)
n_classes = len(classes)

# compute l
r = math.floor(n/2)+1
if(r%2==0):
    r = r+1
l = r

m_test = len(y_tes)
print( m ,m_test,n)
l

43500 14500 9


5

# Training the Sparse RBF Kernel

In [8]:
%%cython
from libc.math cimport exp 

cdef extern from "math.h":
    double sqrt(double m)
    
def sp_rbf(double[:,:] X, double mu,double gamma,double l ):
    cdef list data = []
    cdef list row =[]
    cdef list col =[]
    cdef int i,j,d
    cdef double r = 0
    cdef double yj
    cdef int n1 = X.shape[1]
    cdef int m1 = X.shape[0]
    for i in range(m1):
        for j in range(i,m1):
            r =0
            for d in range(n1):
                    r += (X[i,d] - X[j, d]) ** 2
            yj = 1-mu*sqrt(r)
            if yj>0:
                yj = (yj**l)*exp(-gamma*r) # =data # col = j # row i 
                data.append(yj)
                col.append(j)
                row.append(i)
    return data,row,col



In [20]:
s = time.time()
data,row,col = sp_rbf(x_tr,mu,gamma,l)
K2 = csc_matrix((data, (row, col)), shape=(m, m))
del data,row,col
K2 = K2+K2.transpose()-diags([1],shape = (m,m))    
e = time.time()
print("Time of computing the sparse kernel" ,e-s)
print("sparsity of the kernel" ,1-K2.count_nonzero()/m**2)
total_time = e-s

# Cholesky factorization
f = cholesky(K2, beta=0.0001, mode='auto', ordering_method= 'amd', use_long=None)
Ls1 = f.L()
P = f.P()
print("Sparsity of Matrix L", 1-Ls1.count_nonzero()/m**2)

class2 = False
if(n_classes==2):
    class2 = True
    if(classes[0] == 1):
        pos = y_tr==1
        y_tr[pos] = -1
        neg = y_tr==2
        y_tr[neg] = 1
        pos = y_tes==1
        y_tes[pos] = -1
        neg = y_tes==2
        y_tes[neg] = 1

# LibLinear Classifier
clf = LinearSVC( C = 1,dual = False)
clf.fit(Ls1,y_tr[P])
w = clf.coef_.T
b = clf.intercept_

# Solving for lamdas

if(class2 == True):
    lamda = spsolve_triangular(Ls1.transpose().dot(diags(y_tr[P])), w,lower = False)
else:
    lamda = np.zeros((m,n_classes))
    for i in range(n_classes):
        pos = y_tr==classes[i]
        y_tr_i = -np.ones(m)
        y_tr_i[pos] = 1
        lamda[:,i] = spsolve_triangular(Ls1.transpose().dot(diags(y_tr_i[P])), w[:,i],lower = False)
e = time.time()
total_time = e-s
print("Total Training time ",total_time )

Time of computing the sparse kernel 16.382152318954468
sparsity of the kernel 0.9991811304003171
Sparsity of Matrix L 0.9995790181001454
Total Training time  24.996933698654175


# Testing using Sparse KERNEL

In [14]:
%%cython
from libc.math cimport exp 

cdef extern from "math.h":
    double sqrt(double m)

def sp_rbf_test(double[:,:] X_train,double[:,:] X_test, double mu,double gamma,double l ):
    cdef list data = []
    cdef list row =[]
    cdef list col =[]
    cdef int i,j,d
    cdef double r = 0
    cdef double yj
    cdef int n1 = X_train.shape[1]
    cdef int m_tr = X_train.shape[0]
    cdef int m_tes = X_test.shape[0]
    for i in range(m_tr):
        for j in range(m_tes):
            r =0
            for d in range(n1):
                    r += (X_train[i,d] - X_test[j, d]) ** 2
            yj = 1-mu*sqrt(r)
            if yj>0:
                yj = (yj**l)*exp(-gamma*r) # =data # col = j # row i 
                data.append(yj)
                col.append(j)
                row.append(i)
    return data,row,col

In [21]:
data,row,col = sp_rbf_test(x_tr,x_tes,mu,gamma,l)
K2_t = csc_matrix((data, (row, col)), shape=(m, m_test))
del data,row,col

if(class2 == True):
    y_pred = np.sign(K2_t.T.dot(diags(y_tr).dot(lamda[P])) + b*np.ones((m_test,1)))
else:
    dec_fun = np.zeros((m_test,n_classes))
    lamda = lamda[P]
    for i in range(n_classes):
        pos = y_tr==classes[i]
        y_tr_i = -np.ones(m)
        y_tr_i[pos] = 1
        dec_fun[:,i] = K2_t.T.dot(diags(y_tr_i)).dot(lamda[:,i])  + b[i]*np.ones((m_test))
    I = np.argmax(dec_fun,axis = 1)
    y_pred = classes[I]
acc = accuracy_score(y_pred,y_tes)
print("Accuracy of Sparse kernel",acc)

Accuracy of Sparse kernel 0.8964827586206896


# Training and Testing using RBF kernel in LibSVM

In [16]:
#training and testing using RBF classifier ( LibSVM)
start = time.time()
clf_2 = SVC(C = 1,kernel = 'rbf',gamma = gamma)
clf_2.fit(x_tr,y_tr)
end = time.time()
print("Time of RBF Kernel",end - start)
rbf_acc = clf_2.score(x_tes,y_tes)
print("Accuracy of RBF kernel",rbf_acc)
print("Total Training time ",total_time )
print("Accuracy of Sparse kernel",acc)

Time of RBF Kernel 101.36099028587341
Accuracy of RBF kernel 0.9958620689655172
Total Training time  70.41131258010864
Accuracy of Sparse kernel 0.9166206896551724


# Training and Testing using Liblinear classifier

In [17]:
start = time.time()
clf_3 = LinearSVC(C = 1)
clf_3.fit(x_tr,y_tr)
end = time.time()
print("Time of Linear Kernel",end - start)
lin_acc = clf_3.score(x_tes,y_tes)
print("Accuracy of Linear kernel",lin_acc)

Time of Linear Kernel 2.324842929840088
Accuracy of Linear kernel 0.9229655172413793
